# Search good hyper-parameters in Normalization
- It seems finding a good value helps a lot in performance

In [12]:
import sys

from urllib3.http2.probe import acquire_and_get

sys.path.append("../Share")
import config, utils, baseline, Trainer, Processing_same_with_MATLAB
import os
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

baseline_K_val = [1] #Train ~18 session data, test with 19~
K = baseline_K_val[0]

feature_names = ['Zero Crossing (ZC)', 'Slope Sign Changes (SSC)', 'Waveform Length (WL)', 'WAMP', 'Mean Absolute Value (MAV)', 'Mean Square (MS)', 'Root Mean Square (RMS)',
                 'v-order 3 (V3)', 'log detector (LD)', 'difference absolute standard deviation value (DASDV)', 'maximum fractal length (MFL)', 'myopulse percentage rate (MPR)',
                 'mean absolute value slope (MAVS)', 'weighted mean absolute (WMS)',
                 'Cepstrum Coefficient 1', 'Cepstrum Coefficient 2', 'Cepstrum Coefficient 3', 'Cepstrum Coefficient Average', 'DWTC1', 'DWTC2',
                 'DWTPC1', 'DWTPC2', 'DWTPC3']


feature_idx = range(0,len(feature_names))

fs = round(10e6 / 2048)  # 4883 Hz
lower_cutoff = 100
upper_cutoff = 600
filter_b, filter_a = Processing_same_with_MATLAB.cheby2(4, 30, [lower_cutoff / (fs/2), upper_cutoff / (fs/2)], btype='bandpass')

feat_mean_1ch = np.array([0.1, 0.1, 2.5, 0.0, 11.0, 229.0, 13.8, -11.0, 9.0, 3.0, 1.5, 0.0, 0.0, 2.8])
feat_std_1ch = np.array([0.02, 0.05, 0.65, 0.02, 4.43, 303.9, 6.85, 12.18, 2.87, 0.87, 0.21, 0.04, 6.68, 1.12])
#feat_mean = np.tile(feat_mean_1ch, (4, 1))
#feat_std = np.tile(feat_std_1ch, (4, 1))

feat_mean_1ch = np.array([0.1, 0.1,
                          [2, 2.5, 3],
                          0.0, 11.0, 229.0, 13.8, -11.0, 9.0, 3.0, 1.5, 0.0, 0.0, 2.8])
feat_std_1ch = np.array([0.02, 0.05,
                         [0,5, 0.65, 0.8],
                         0.02, 4.43, 303.9, 6.85, 12.18, 2.87, 0.87, 0.21, 0.04, 6.68, 1.12])

SUBJECT = "Carlson"
data_files = config.dataset_sub_C
default_path = config.default_path_sub_C

trainer = Trainer.TremorModelTrainer(config, subject=SUBJECT)

In [16]:
import importlib
importlib.reload(Processing_same_with_MATLAB)

#1D - one feature at a time
def Train_and_test(Normalization_TF, feat_mean_1ch, feat_std_1ch, num_feature_set, target_feat_idx):

    extractor = Processing_same_with_MATLAB.EMGFeatureExtractor(feat_mean_1ch, feat_std_1ch, filter_b, filter_a, Norm_bool=Normalization_TF, num_feature_set=num_feature_set)
    X_train_all, y_train_all, X_test_all, y_test_all, X_val_all, y_val_all = [], [], [], [], [], []
    unseen_test_result = []

    for idx, session_info in enumerate(data_files):
        print(f"Dataset {idx + 1}/{len(data_files)} - Session {session_info}\n{'='*40}")
        path = os.path.join(default_path, f'{session_info}raw/')
        features, class_labels = [], []
        for c_idx, c in enumerate(config.classes_5):
            raw_data = os.listdir(path+c)
            mat = scipy.io.loadmat(path+c+raw_data[0])
            extractor.buffer = mat['Data_ADC']
            class_labels.append(mat['Data_Cls'].reshape(-1))

            #### features_per_cls = extractor.extract_features(num_feature_set=num_feature_set)  ### 이건 2D input
            features_per_cls = extractor.extract_one_feature_at_a_time(target_feature_idx=target_feat_idx)  ###1D input

            #여기서 Normalization_TF
            #features = (features - self.feat_mean[:, :, np.newaxis]) / self.feat_std[:, :, np.newaxis]
            features_per_cls = extractor.Normalization(features_per_cls, target_feat_idx)

            features_per_cls = np.transpose(features_per_cls, (1, 0))  # shape: (1729, 4, 14)
            features.append(features_per_cls)
            #print(features_per_cls.shape, mat['Data_Cls'].reshape(-1).shape)

        X = np.concatenate(features, axis=0)
        y = np.concatenate(class_labels, axis=0)
        if X.shape[0] != y.shape[-1]:
            print(f"Incorrect shape between features and Class: {X.shape} and {y.shape}, {session_info}")
            break

        if idx < K:
            X_train, y_train, X_val, y_val = utils.split_data(X, y, ratio=0.8)
            X_train_all.append(X_train)
            y_train_all.append(y_train)
            X_val_all.append(X_train)
            y_val_all.append(y_train)

        elif idx == K:
            X_train, y_train, X_test, y_test,  = utils.split_data(X, y, ratio=0.8)
            X_train_all.append(X_train)
            y_train_all.append(y_train)
            X_val_all.append(X_test)
            y_val_all.append(y_test)

            X_train_stacked = np.concatenate(X_train_all, axis=0)
            y_train_stacked = np.concatenate(y_train_all, axis=0)
            print(f"\t Training {K}: ", X_train_stacked.shape, y_train_stacked.shape)
            acc, pre_trained_CNN = trainer.train_multiple_dataset_1D(X_train, y_train, X_test, y_test)
            print(f"\t Accuracy on test dataset {idx+1}: {acc:.4f}%")

        else:
            X_test, y_test, _, _ = utils.split_data(X, y, ratio=1)
            X_test_all.append(X_test)
            y_test_all.append(y_test)
            X_test_stacked = np.concatenate(X_test_all, axis=0)
            y_test_stacked = np.concatenate(y_test_all, axis=0)

            X = np.expand_dims(X, axis=-1)
            acc = pre_trained_CNN.evaluate(X_test, y_test, verbose=0)[1]*100
            print(f"\t Accuracy on unseen dataset {idx+1}: {acc:.4f}%")
            unseen_test_result.append(acc)

    return unseen_test_result, X_train_stacked, y_train_stacked, X_test_stacked, y_test_stacked

In [17]:
Normalization_TF = True ############################ important!!!!
feature_idx = 2 #WL
test_acc, X_train_stacked, y_train_stacked, X_test_stacked, y_test_stacked = Train_and_test(Normalization_TF, feat_mean_1ch, feat_std_1ch,
                                                                                            num_feature_set=14, target_feat_idx=feature_idx)
print(X_train_stacked.shape, y_train_stacked.shape, X_test_stacked.shape, y_test_stacked.shape)

Dataset 1/8 - Session Exp_2025-06-30-v1/E9AD0E7DCC2B/
Dataset 2/8 - Session Exp_2025-06-30-v2/E9AD0E7DCC2B/
	 Training 1:  (13641, 4, 1) (13641,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 85.76%
Maximum validation accuracy : 87.32%
Accuracy of test dataset using model V0: 87.1386%
	 Accuracy on test dataset 2: 87.3156%
Dataset 3/8 - Session Exp_2025-07-09-v1/E9AD0E7DCC2B/
	 Accuracy on unseen dataset 3: 85.3961%
Dataset 4/8 - Session Exp_2025-07-09-v2/E9AD0E7DCC2B/
	 Accuracy on unseen dataset 4: 84.0135%
Dataset 5/8 - Session Exp_2025-07-10-v1/E9AD0E7DCC2B/
	 Accuracy on unseen dataset 5: 81.9119%
Dataset 6/8 - Session Exp_2025-07-10-v2/E9AD0E7DCC2B/
	 Accuracy on unseen dataset 6: 83.2542%
Dataset 7/8 - Session Exp_2025-07-11-v1/E9AD0E7DCC2B/
	 Accuracy on unseen dataset 7: 84.8176%
Dataset 8/8 - Session Exp_2025-07-11-v2/E9AD0E7DCC2B/
	 Accuracy on unseen dataset 8: 84.6091%
(13641, 4, 1) (13641,) (51743, 4, 1) (51743,)


In [10]:
test_acc #Without Norm

[83.41259956359863,
 84.05981659889221,
 78.71378660202026,
 81.50550127029419,
 82.3972225189209,
 82.63145685195923]

In [18]:
test_acc #With Norm

[85.39612293243408,
 84.01345014572144,
 81.91193342208862,
 83.25420022010803,
 84.81760025024414,
 84.60912108421326]

In [ ]:
이제 좋은 param 찾기

In [ ]:
근데 다 같이 했을때 Normalization이 중요한거고, 다 같이 안하면 크게 상관없는 거 아닌가요
- 아니지, 일단 좋은 param을 찾고 그거를 합치면 더 좋아질듯

In [ ]:
# (13641, 4, 18, 1) (13641,) (51743, 4, 18, 1) (51743,) => X_train, y_train, X_test, y_test shape

def feature_wise_norm_search(X_train_stacked, y_train_stacked, X_test_stacked, y_test_stacked,
                            feature_idx,  # list of feature indices to test (e.g., range(14))
                            mean_candidates,  # list of means to try (e.g., [0.0, 0.1, 0.5, 1.0])
                            std_candidates,   # list of stds to try (e.g., [0.1, 0.5, 1.0, 2.0]),
                            trainer
                        ):
    best_mean_std_list = []
    feature_acc = []

    for f_idx in feature_idx:
        print(f"🔍 Searching normalization params for feature {f_idx}...")
        best_acc = -1
        best_mean = None
        best_std = None

        # Extract the single feature → shape: (N, 4, 1)
        X_train_f = X_train_stacked[:, :, f_idx:f_idx+1, :]
        X_test_f = X_test_stacked[:, :, f_idx:f_idx+1, :]

        X_train_f = np.squeeze(X_train_f, axis=-1)  # (N, 4, 1)
        X_test_f = np.squeeze(X_test_f, axis=-1)
        print(X_train_f.shape, X_test_f.shape)

        for mean in mean_candidates:
            for std in std_candidates:
                if std < 1e-6:  # Avoid division by zero
                    continue

                # Normalize manually
                X_train_norm = (X_train_f - mean) / std
                X_test_norm = (X_test_f - mean) / std

                # Train and evaluate
                acc, _ = trainer.train_multiple_dataset_1D(
                    X_train_norm, y_train_stacked,
                    X_test_norm, y_test_stacked
                )

                if acc > best_acc:
                    best_acc = acc
                    best_mean = mean
                    best_std = std

        print(f"✅ Best for feature {f_idx}: mean={best_mean}, std={best_std}, acc={best_acc:.4f}")
        best_mean_std_list.append((best_mean, best_std))
        feature_acc.append(best_acc)

    return best_mean_std_list, feature_acc



In [ ]:
feat_mean_1ch = np.array([0.1, 0.1, 2.5, 0.0, 11.0, 229.0, 13.8, -11.0, 9.0, 3.0, 1.5, 0.0, 0.0, 2.8])
feat_std_1ch = np.array([0.02, 0.05, 0.65, 0.02, 4.43, 303.9, 6.85, 12.18, 2.87, 0.87, 0.21, 0.04, 6.68, 1.12])
feat_mean = np.tile(feat_mean_1ch, (4, 1))
feat_std = np.tile(feat_std_1ch, (4, 1))


feature_idx = [2] #WL (좋았음)
mean_candidates_lst = [2.5]#[2, 2.5, 3]
std_candidates_lst = [0.65]#[0.5, 0.65, 0.8]

print(feature_names[feature_idx[0]])
best_mean_std_list, feature_acc = feature_wise_norm_search(X_train_stacked, y_train_stacked, X_test_stacked, y_test_stacked,
                                                           feature_idx, mean_candidates_lst, std_candidates_lst, trainer)


In [ ]:
feature_vis(feature_names, feature_acc_Norm)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(Norm_acc, label="Without Normalization", marker='o')
plt.plot(noNorm_acc, label="With Normalization", marker='s')
plt.title("Feature Accuracy Comparison (Normalized vs. Not Normalized)")
plt.xlabel("Feature Index")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(feature_acc_noNorm, label="Without Normalization", marker='o')
plt.plot(feature_acc_Norm, label="With Normalization", marker='s')
plt.title("Feature Accuracy Comparison (Normalized vs. Not Normalized)")
plt.xlabel("Feature Index")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Conclusion
- 피쳐 하나씩 돌릴때는 Normalization 안해도 큰 손실은 없음
- 여러개 돌릴때는 Normalization 중요
- 비슷하다면 Normalization 을 하고 돌리고 거기서 좋은 값 찾기